<a target="_blank" href="https://colab.research.google.com/github/cerr/pyCERR-Notebooks/blob/main/ai_seg_inference_ex1.ipynb">
  <img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/>
</a>

## AI inference on images in planC

This notebook demonstrates the use of pyCERR in AI inference. planC keeps track of associations between various imaging objects. These objects can then be passed to an inference routine, e.g. image segmentation, registration, outcomes modeling. Example below demonstrates the steps involved in using image from planC as an input to segmentation model and then exporting the result to DICOM.

#### Install pyCERR

In [ ]:
%%capture
!pip install git+https://github.com/cerr/pyCERR/

#### Get path to example dicom dataset in pyCERR

In [ ]:
from cerr import datasets
import os
phantom_dir = os.path.join(os.path.dirname(datasets.__file__),'radiomics_phantom_dicom')
pat_names = ['PAT1', 'PAT2', 'PAT3', 'PAT4']
all_pat_dirs = [os.path.join(phantom_dir, pat) for pat in pat_names]
dcm_dir = all_pat_dirs[0]

#### Import dicom to planC

In [ ]:
from cerr import plan_container as pc
planC = pc.load_dcm_dir(dcm_dir)

#### Export the desired image to nii or pass numpy array to inference wrapper

In [ ]:
import tempfile
import os
niiExportDir = tempfile.gettempdir()
scanNum = 0 # index of scan to export to nii
str_num = 0 # index of segmentation to export to nii
scanNiiFile = os.path.join(niiExportDir,'scan_from_cerr.nii.gz')
planC.scan[scanNum].save_nii(scanNiiFile)
str_name = planC.structure[str_num].structureName
strNiiFile = os.path.join(niiExportDir,str_name+'.nii.gz')
planC.structure[str_num].save_nii(strNiiFile,planC)
print("Exported nii files to " + niiExportDir)

#### Read segmentation output as nii

In [ ]:
niiSegOutputFile = strNiiFile # same as input segmentation for now. This will be output of an AI model
assocScanNum = 0
labelsDict = {1: "test_ai_struct"}
planC = pc.load_nii_structure(niiSegOutputFile, assocScanNum, planC, labelsDict)


#### Export segmentation to dicom

In [ ]:
from cerr.dcm_export import rtstruct_iod
rtstructFile = os.path.join(niiExportDir,"ai_seg_test.dcm")
structNumV = [len(planC.structure)-1] # Export the last structure in the list
seriesDescription = "AI Generated"
rtstruct_iod.create(structNumV,rtstructFile,planC,seriesDescription)


#### Clean-up exported files

In [ ]:
os.remove(scanNiiFile)
os.remove(strNiiFile)
os.remove(rtstructFile)